* Make some examples for allowing ICL
    * Take annotated examples and add expected answers 

In [1]:
import pandas as pd
from tqdm import tqdm
import re

In [2]:
ANNOT_DATA = "./Reports/example_plausible_gallstones.csv"

In [3]:
# ## Example:

## "You are a helpful scientific researcher. You are critical and grounded in scientific evidence, while being open to new connections and novel leaps. You research, understand, filter and validate (or disprove) candidate novel discoveries and features for research"

# # # Define a few examples of human annotations
# icl_examples = [
#     {"role": "user", "content": "Considering the feature 'High blood pressure' (p-value: 0.01, importance: 0.5, correlation: 0.45) related to biomedical disease understanding of 'Cardiovascular Diseases', is this interesting?"},
#     {"role": "assistant", "content": "Yes, because high blood pressure is a well-known risk factor for cardiovascular diseases, and its significant p-value and moderate importance suggest it has predictive value in this context."},
#     {"role": "user", "content": "Given the feature 'Daily sugar intake' (p-value: 0.15, importance: 0.1, correlation: 0.05) related to biomedical and clinical healthcare research of 'Diabetes', does this make sense?"},
#     {"role": "assistant", "content": "No, despite the known relationship between sugar intake and diabetes, the high p-value and low importance indicate that it may not be a strong predictor in this dataset."}
# ]

# def query_gpt(prompt):
#     """Function to query GPT-4 model and return the response, with error handling."""
#     try:
#         messages = [
#             {"role": "system", "content": "You are a helpful research assistant. You are also highly critical, but only when grounded in scientific or prior evidence."},
#             *icl_examples,  # Add ICL examples to each query
#             {"role": "user", "content": prompt}
#         ]
#         response = openai.ChatCompletion.create(
#             model="gpt-4o-mini",
#             messages=messages,
#             max_tokens=256,
#             temperature=0.95,
#             stop=None
#         )
#         return response.choices[0].message['content'].strip()
#     except Exception as e:
#         print(f"An error occurred: {e}")
#         return "Error in generating response."


In [5]:
df = pd.read_csv(ANNOT_DATA)
df

,feature_name,feature_importance,p_val,correlation,MutualInfoTarget,Co-occurrence Count,Target,Interesting?,Plausible/Makes Sense?,COMMENTS
0,"Long-standing illness, disability or infirmity...",0.041,0.000,0.049,NaN,0,(Cholelithiasis) OR (Gallstones) OR (Gallbladd...,True,True,CVD risk factors
1,Alanine aminotransferase,0.032,0.000,0.027,NaN,0,(Cholelithiasis) OR (Gallstones) OR (Gallbladd...,False,True,liver enzyme
2,Mouth/teeth dental problems Dentures,0.023,0.000,0.031,NaN,0,(Cholelithiasis) OR (Gallstones) OR (Gallbladd...,True,True,nutrition
3,Overall health rating Fair,0.016,0.000,0.043,NaN,0,(Cholelithiasis) OR (Gallstones) OR (Gallbladd...,False,False,NaN
4,Standard PRS for alzheimer's disease (AD),0.012,0.000,-0.008,NaN,0,(Cholelithiasis) OR (Gallstones) OR (Gallbladd...,True,True,NaN
...,...,...,...,...,...,...,...,...,...,...
79,3-Hydroxybutyrate,0.001,0.306,-0.002,0.0,0,(Cholelithiasis) OR (Gallstones) OR (Gallbladd...,False,False,High P value + low correlation
80,Hallux valgus (Bunion),0.001,0.095,0.000,0.0,0,(Cholelithiasis) OR (Gallstones) OR (Gallbladd...,False,False,High P value + low correlation
81,Cholesterol in Chylomicrons and Extremely Larg...,0.001,0.000,0.002,0.0,0,(Cholelithiasis) OR (Gallstones) OR (Gallbladd...,True,False,- Our meta-analysis found insufficient eviden...
82,Job code at visit clerks,0.001,0.080,-0.003,0.0,0,(Cholelithiasis) OR (Gallstones) OR (Gallbladd...,False,False,This is a not an ordinal data (i assume) - pr...


In [6]:
print(df.loc[df["p_val"]>0.5].shape[0]) # we might want the model to learn from these, or just drop them
df= df.loc[(df["p_val"]<0.5) | (df["feature_importance"]>=0.01)]
print(df.shape[0])

12
72


In [7]:
# df = df.loc[df["COMMENTS"].str.strip()!="High P value"]
## skip these cases
df = df.loc[~df["COMMENTS"].str.strip().str.contains("High P value|same feature",case=False,na=False)]
df

,feature_name,feature_importance,p_val,correlation,MutualInfoTarget,Co-occurrence Count,Target,Interesting?,Plausible/Makes Sense?,COMMENTS
0,"Long-standing illness, disability or infirmity...",0.041,0.000,0.049,NaN,0,(Cholelithiasis) OR (Gallstones) OR (Gallbladd...,True,True,CVD risk factors
1,Alanine aminotransferase,0.032,0.000,0.027,NaN,0,(Cholelithiasis) OR (Gallstones) OR (Gallbladd...,False,True,liver enzyme
2,Mouth/teeth dental problems Dentures,0.023,0.000,0.031,NaN,0,(Cholelithiasis) OR (Gallstones) OR (Gallbladd...,True,True,nutrition
3,Overall health rating Fair,0.016,0.000,0.043,NaN,0,(Cholelithiasis) OR (Gallstones) OR (Gallbladd...,False,False,NaN
4,Standard PRS for alzheimer's disease (AD),0.012,0.000,-0.008,NaN,0,(Cholelithiasis) OR (Gallstones) OR (Gallbladd...,True,True,NaN
...,...,...,...,...,...,...,...,...,...,...
74,Alcohol-related disorders,0.002,0.051,0.001,0.0,0,(Cholelithiasis) OR (Gallstones) OR (Gallbladd...,True,False,oppsite to https://pubmed.ncbi.nlm.nih.gov/279...
75,Alcohol intake frequency. Never,0.002,0.000,-0.002,0.0,0,(Cholelithiasis) OR (Gallstones) OR (Gallbladd...,True,False,oppsite to https://pubmed.ncbi.nlm.nih.gov/279...
77,Qualifications College or University degree|A ...,0.002,0.000,-0.003,0.0,0,(Cholelithiasis) OR (Gallstones) OR (Gallbladd...,False,False,younger people? is this an ordinal value?
81,Cholesterol in Chylomicrons and Extremely Larg...,0.001,0.000,0.002,0.0,0,(Cholelithiasis) OR (Gallstones) OR (Gallbladd...,True,False,- Our meta-analysis found insufficient eviden...


#### Maybe keep cases without comments as "Test set" (they are annotated after all). 

In [8]:
df.loc[df["COMMENTS"].isna()].shape[0]

22

* Warning/note: We are skipping cases without comments. And worse, we are not seperating explanation types (e.g. for something being interesting/novel vs plausible).

In [9]:
DROP_UNCOMMENTED_EXAMPLES = True#False

In [10]:
data = df.drop_duplicates(subset=df.select_dtypes(["O","bool"]).columns)
print(data.shape[0])
if DROP_UNCOMMENTED_EXAMPLES:
    data = data.loc[~data["COMMENTS"].isna()].copy()
    print(data.shape[0])
#Drop more cases - context limit
data = data.loc[~data['COMMENTS'].str.contains("P val|p-val|correlation|ordinal|similar to",case=False,na=False)]
print(data.shape[0])
# data = data.drop_duplicates(subset=['feature_name']).drop_duplicates(subset=['COMMENTS']) ## 31 -> 27
data = data.drop_duplicates(subset=['feature_name','COMMENTS']) ## 
print(data.shape)

65
43
26
(26, 10)


In [11]:
data.loc[data['COMMENTS'].str.split().str.len()<2] # # words is not a good filter

,feature_name,feature_importance,p_val,correlation,MutualInfoTarget,Co-occurrence Count,Target,Interesting?,Plausible/Makes Sense?,COMMENTS
2,Mouth/teeth dental problems Dentures,0.023,0.0,0.031,NaN,0,(Cholelithiasis) OR (Gallstones) OR (Gallbladd...,True,True,nutrition
6,Haemoglobin concentration,0.010,0.0,-0.014,NaN,0,(Cholelithiasis) OR (Gallstones) OR (Gallbladd...,False,True,M<>F
7,SHBG,0.010,0.0,-0.006,0.000,0,(Cholelithiasis) OR (Gallstones) OR (Gallbladd...,False,True,M<>F
8,Standing height,0.009,0.0,-0.041,NaN,0,(Cholelithiasis) OR (Gallstones) OR (Gallbladd...,False,True,M<>F
11,Townsend deprivation index at recruitment,0.006,0.0,0.024,NaN,0,(Cholelithiasis) OR (Gallstones) OR (Gallbladd...,True,True,important
35,Apolipoprotein A1,0.010,0.0,0.006,0.001,0,(Cholelithiasis) OR (Gallstones) OR (Gallbladd...,True,False,https://www.sciencedirect.com/science/article/...


In [12]:
data.nunique()

feature_name              26
feature_importance        15
p_val                      4
correlation               19
MutualInfoTarget           6
Co-occurrence Count        1
Target                     1
Interesting?               2
Plausible/Makes Sense?     2
COMMENTS                  22
dtype: int64

In [44]:
def get_examples(data,target,get_interesting=True,get_plausible=True,print_plausible_prompt=False):
    results = []
    for index, row in tqdm(data.iterrows(), total=data.shape[0], desc="Processing rows"):
        feature_name = row['feature_name']
        target = row['Target'].replace(r"\(\)","").replace(" AND "," ").replace(r"(","").replace(r")","")
        # target = "Cholelithiasis OR Gallstones OR Gallbladder disease OR cholecystitis"
        p_val = row['p_val']
        feature_importance = row['feature_importance']
        correlation = row['correlation']
        ## related to biomedical and healthcare disease understanding, prognosis, research, and prediction of:
        prompt_interesting = f"Given the feature '{feature_name}'; (p-value: {p_val}, correlation: {correlation}, SHAP importance {feature_importance}), as a potential predictive feature, or risk/protective factor for target disease: '{target}', is this feature interesting and Novel? Predict 'Yes.' or 'No.', then explain why for a medical expert."
        prompt_plausible = f"Given the feature '{feature_name}'; (p-value: {p_val}, correlation: {correlation}, SHAP importance {feature_importance}), as a potential predictive feature, or risk/protective factor for target disease:'{target}', does this feature make sense? Is there a plausible mechanism for the feature to affect the target disease? Predict 'Yes.' or 'No.', then explain for a medical expert."
        if print_plausible_prompt:
            print(prompt_plausible)
            print()
        response_interesting = f"{'Yes' if row['Interesting?'] else 'No'}. {'' if pd.isna(row['COMMENTS']) else row['COMMENTS'].strip()}"
        # response_plausible = f"{'Yes' if row['Plausible/Makes Sense?'] else 'No'}. {'' if pd.isna(row['COMMENTS']) else row['COMMENTS'].strip()}"
    
        # response_interesting = f"{'Yes' if row['Interesting?'] else:'No'}. {row['COMMENTS'].str.strip()}" ## orig
    
        response_plausible = f"{'Yes' if row['Plausible/Makes Sense?'] else 'No'}."
        # # if not row['COMMENTS'].str.contains("P value|correlation",case=False,na=False):
        # if not row['COMMENTS'].contains("P value|correlation",case=False,na=False):
            # response_plausible = response_plausible + f" {row['COMMENTS'].str.strip()}"
        # Check if 'COMMENTS' does not contain "P value" or "correlation"
        if pd.notna(row['COMMENTS']) and not re.search("P value|correlation", row['COMMENTS'], flags=re.IGNORECASE):
            response_plausible += f" {row['COMMENTS'].strip()}"    
        if get_interesting:
            results.append( {"role": "user", "content":prompt_interesting})
            results.append( {"role": "assistant", "content":response_interesting})
        if get_plausible:
            results.append( {"role": "user", "content":prompt_plausible})
            results.append( {"role": "assistant", "content":response_plausible})
    return results

In [14]:
results = get_examples(data,target = "Cholelithiasis OR Gallstones OR Gallbladder disease OR cholecystitis")

Processing rows: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 26/26 [00:00<00:00, 17331.83it/s]


In [15]:
data

,feature_name,feature_importance,p_val,correlation,MutualInfoTarget,Co-occurrence Count,Target,Interesting?,Plausible/Makes Sense?,COMMENTS
0,"Long-standing illness, disability or infirmity...",0.041,0.000,0.049,NaN,0,(Cholelithiasis) OR (Gallstones) OR (Gallbladd...,True,True,CVD risk factors
1,Alanine aminotransferase,0.032,0.000,0.027,NaN,0,(Cholelithiasis) OR (Gallstones) OR (Gallbladd...,False,True,liver enzyme
2,Mouth/teeth dental problems Dentures,0.023,0.000,0.031,NaN,0,(Cholelithiasis) OR (Gallstones) OR (Gallbladd...,True,True,nutrition
5,Overall health rating Excellent,0.011,0.000,-0.042,NaN,0,(Cholelithiasis) OR (Gallstones) OR (Gallbladd...,True,True,healthy life style
6,Haemoglobin concentration,0.010,0.000,-0.014,NaN,0,(Cholelithiasis) OR (Gallstones) OR (Gallbladd...,False,True,M<>F
7,SHBG,0.010,0.000,-0.006,0.000,0,(Cholelithiasis) OR (Gallstones) OR (Gallbladd...,False,True,M<>F
8,Standing height,0.009,0.000,-0.041,NaN,0,(Cholelithiasis) OR (Gallstones) OR (Gallbladd...,False,True,M<>F
9,Gastritis and duodenitis,0.009,0.000,0.013,NaN,0,(Cholelithiasis) OR (Gallstones) OR (Gallbladd...,True,True,life style
10,Standard PRS for venous thromboembolic disease...,0.008,0.002,0.007,NaN,0,(Cholelithiasis) OR (Gallstones) OR (Gallbladd...,False,True,Obesity and lifesytle
11,Townsend deprivation index at recruitment,0.006,0.000,0.024,NaN,0,(Cholelithiasis) OR (Gallstones) OR (Gallbladd...,True,True,important


In [45]:
# res1 = get_examples(data.loc[data['Interesting?'] & ~data['Plausible/Makes Sense?']],
#              target = "Cholelithiasis OR Gallstones OR Gallbladder disease OR cholecystitis",get_interesting=False)

res1 = get_examples(data.loc[data['Interesting?'] & data['Plausible/Makes Sense?']],
             target = "Cholelithiasis OR Gallstones OR Gallbladder disease OR cholecystitis",get_interesting=False,print_plausible_prompt=True)

Processing rows: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 5312.35it/s]

Given the feature 'Long-standing illness, disability or infirmity Yes'; (p-value: 0.0, correlation: 0.049, SHAP importance 0.041), as a potential predictive feature, or risk/protective factor for target disease:'Cholelithiasis OR Gallstones OR Gallbladder disease OR cholecystitis OR Cholangitis', does this feature make sense? Is there a plausible mechanism for the feature to affect the target disease? Predict 'Yes.' or 'No.', then explain for a medical expert.

Given the feature 'Mouth/teeth dental problems Dentures'; (p-value: 0.0, correlation: 0.031, SHAP importance 0.023), as a potential predictive feature, or risk/protective factor for target disease:'Cholelithiasis OR Gallstones OR Gallbladder disease OR cholecystitis OR Cholangitis', does this feature make sense? Is there a plausible mechanism for the feature to affect the target disease? Predict 'Yes.' or 'No.', then explain for a medical expert.

Given the feature 'Overall health rating Excellent'; (p-value: 0.0, correlation: -

In [40]:
len(res1)

26

In [46]:
# print([res1[i]["content"] for i in range(len(res1))])
for i in range(len(res1)):
    print(res1[i]["content"],"\n")

Given the feature 'Long-standing illness, disability or infirmity Yes'; (p-value: 0.0, correlation: 0.049, SHAP importance 0.041), as a potential predictive feature, or risk/protective factor for target disease:'Cholelithiasis OR Gallstones OR Gallbladder disease OR cholecystitis OR Cholangitis', does this feature make sense? Is there a plausible mechanism for the feature to affect the target disease? Predict 'Yes.' or 'No.', then explain for a medical expert. 

Yes. CVD risk factors 

Given the feature 'Mouth/teeth dental problems Dentures'; (p-value: 0.0, correlation: 0.031, SHAP importance 0.023), as a potential predictive feature, or risk/protective factor for target disease:'Cholelithiasis OR Gallstones OR Gallbladder disease OR cholecystitis OR Cholangitis', does this feature make sense? Is there a plausible mechanism for the feature to affect the target disease? Predict 'Yes.' or 'No.', then explain for a medical expert. 

Yes. nutrition 

Given the feature 'Overall health ratin

In [24]:
get_examples(data.loc[data['Interesting?'] & data['Plausible/Makes Sense?']],
             target = "Cholelithiasis OR Gallstones OR Gallbladder disease OR cholecystitis",get_interesting=False)

Processing rows: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 6382.53it/s]


[{'role': 'user',
  'content': "Given the feature 'Long-standing illness, disability or infirmity Yes'; (p-value: 0.0, correlation: 0.049, SHAP importance 0.041), as a potential predictive feature, or risk/protective factor for target disease:'Cholelithiasis OR Gallstones OR Gallbladder disease OR cholecystitis OR Cholangitis', does this feature make sense? Is there a plausible mechanism for the feature to affect the target disease? Predict 'Yes.' or 'No.', then explain for a medical expert."},
 {'role': 'assistant', 'content': 'Yes. CVD risk factors'},
 {'role': 'user',
  'content': "Given the feature 'Mouth/teeth dental problems Dentures'; (p-value: 0.0, correlation: 0.031, SHAP importance 0.023), as a potential predictive feature, or risk/protective factor for target disease:'Cholelithiasis OR Gallstones OR Gallbladder disease OR cholecystitis OR Cholangitis', does this feature make sense? Is there a plausible mechanism for the feature to affect the target disease? Predict 'Yes.' or